In [1]:
import tensorflow as tf
import pandas as pd
import pickle
from sklearn.utils import shuffle
from sklearn.metrics import mean_absolute_error
import csv

In [2]:

"""
Global values
"""
folder = 'boxoffice_10'

len_dtcgum = pickle.load(open('../data/'+folder+'/len_dtcgum.pkl', 'rb'))

FEATURE_LEN = 5

MOVIE_NUM = len_dtcgum[5]
DIRECTOR_NUM = len_dtcgum[0]
GENRE_NUM = len_dtcgum[3]
TOPIC_NUM = 50
CAST_NUM = len_dtcgum[2]
USER_NUM = len_dtcgum[4]

print('movie:', MOVIE_NUM)
print('user:', USER_NUM)
print('director:', DIRECTOR_NUM)
print('genre:', GENRE_NUM)
print('topic:', TOPIC_NUM)
print('cast:', CAST_NUM)

"""
MOVIE_NUM = 1682
DIRECTOR_NUM = 1139
GENRE_NUM = 24
TOPIC_NUM = 50
CAST_NUM = 2894
USER_NUM = 943
"""

movie: 206
user: 3550
director: 194
genre: 20
topic: 50
cast: 474


'\nMOVIE_NUM = 1682\nDIRECTOR_NUM = 1139\nGENRE_NUM = 24\nTOPIC_NUM = 50\nCAST_NUM = 2894\nUSER_NUM = 943\n'

In [3]:
USER_NUM

3550

In [4]:
movie_rating = pickle.load(open('../data/'+folder+'/movie_rating.pkl', 'rb'))
movie_rating_test = pickle.load(open('../data/'+folder+'/movie_rating_test.pkl', 'rb'))
movie_rating_validation = pickle.load(open('../data/'+folder+'/movie_rating_validation.pkl', 'rb'))


In [5]:
def load_data(user_name, movie_name, rating_name, last_same):
    user = pickle.load(open('../data/'+folder+'/'+user_name+last_same+'.pkl', 'rb'))
    movie = pickle.load(open('../data/'+folder+'/'+movie_name+last_same+'.pkl', 'rb'))
    rating = pickle.load(open('../data/'+folder+'/'+rating_name+last_same+'.pkl', 'rb'))
    return shuffle(user, movie, rating)

#load training data
d_user, d_movie, d_rating = load_data("train_user", "train_movie", "train_rating", '_d')
t_user, t_movie, t_rating = load_data("train_user", "train_movie", "train_rating", '_t')
g_user, g_movie, g_rating = load_data("train_user", "train_movie", "train_rating", '_g')   
c_user, c_movie, c_rating = load_data("train_user", "train_movie", "train_rating", '_c')

#load validation data
d_user_v, d_movie_v, d_rating_v = load_data("train_user", "train_movie", "train_rating", '_d_v')
t_user_v, t_movie_v, t_rating_v = load_data("train_user", "train_movie", "train_rating", '_t_v')
g_user_v, g_movie_v, g_rating_v = load_data("train_user", "train_movie", "train_rating", '_g_v')        
c_user_v, c_movie_v, c_rating_v = load_data("train_user", "train_movie", "train_rating", '_c_v')


In [6]:
U = tf.Variable(initial_value=tf.truncated_normal([USER_NUM, FEATURE_LEN]), name='users')
D = tf.Variable(initial_value=tf.truncated_normal([FEATURE_LEN, DIRECTOR_NUM]), name='directors')
T = tf.Variable(initial_value=tf.truncated_normal([FEATURE_LEN, TOPIC_NUM]), name='topics')
G = tf.Variable(initial_value=tf.truncated_normal([FEATURE_LEN, GENRE_NUM]), name='genures')
C = tf.Variable(initial_value=tf.truncated_normal([FEATURE_LEN, CAST_NUM]), name='casts')

In [7]:
d_result = tf.matmul(U, D)
t_result = tf.matmul(U, T)
g_result = tf.matmul(U, G)
c_result = tf.matmul(U, C)

In [8]:
clip_U = U.assign(tf.maximum(tf.zeros_like(U), U))
clip_D = D.assign(tf.maximum(tf.zeros_like(D), D))
clip_T = T.assign(tf.maximum(tf.zeros_like(T), T))
clip_C = C.assign(tf.maximum(tf.zeros_like(C), C))
clip_G = G.assign(tf.maximum(tf.zeros_like(G), G))
clip = tf.group(clip_U, clip_D,clip_T,clip_C,clip_G )

In [9]:
g_result

<tf.Tensor 'MatMul_2:0' shape=(3550, 20) dtype=float32>

In [10]:
d_result_flatten = tf.reshape(d_result, [-1])
t_result_flatten = tf.reshape(t_result, [-1])
g_result_flatten = tf.reshape(g_result, [-1])
c_result_flatten = tf.reshape(c_result, [-1])

D_R = tf.gather(d_result_flatten, d_user * tf.shape(d_result)[1] + d_movie, name='director_user_rate')
T_R = tf.gather(t_result_flatten, t_user * tf.shape(t_result)[1] + t_movie, name='topic_user_rate')
G_R = tf.gather(g_result_flatten, g_user * tf.shape(g_result)[1] + g_movie, name='genre_user_rate')
C_R = tf.gather(c_result_flatten, c_user * tf.shape(c_result)[1] + c_movie, name='cast_user_rate')

In [11]:
d_diff_op = tf.subtract(D_R, d_rating, name='d_trainig_diff')
t_diff_op = tf.subtract(T_R, t_rating, name='t_trainig_diff')
g_diff_op = tf.subtract(G_R, g_rating, name='g_trainig_diff')
c_diff_op = tf.subtract(C_R, c_rating, name='c_trainig_diff')

d_diff_op_squared = tf.abs(d_diff_op, name="d_squared_difference")
t_diff_op_squared = tf.abs(t_diff_op, name="t_squared_difference")
g_diff_op_squared = tf.abs(g_diff_op, name="g_squared_difference")
c_diff_op_squared = tf.abs(c_diff_op, name="c_squared_difference")

#"""
d_base_cost = tf.reduce_sum(d_diff_op_squared, name="d_sum_squared_error")/ tf.cast(tf.shape(d_diff_op_squared)[0], tf.float32) * 10000 
t_base_cost = tf.reduce_sum(t_diff_op_squared, name="t_sum_squared_error")/ tf.cast(tf.shape(t_diff_op_squared)[0], tf.float32) * 10000
g_base_cost = tf.reduce_sum(g_diff_op_squared, name="g_sum_squared_error")/ tf.cast(tf.shape(g_diff_op_squared)[0], tf.float32) * 10000
c_base_cost = tf.reduce_sum(c_diff_op_squared, name="c_sum_squared_error")/ tf.cast(tf.shape(c_diff_op_squared)[0], tf.float32) * 10000

"""

d_base_cost = tf.reduce_sum(d_diff_op_squared, name="d_sum_squared_error")/30.
t_base_cost = tf.reduce_sum(t_diff_op_squared, name="t_sum_squared_error")/30.
g_base_cost = tf.reduce_sum(g_diff_op_squared, name="g_sum_squared_error")/30.
c_base_cost = tf.reduce_sum(c_diff_op_squared, name="c_sum_squared_error")/30.
"""

base_cost = d_base_cost + t_base_cost + g_base_cost + c_base_cost


In [12]:
lda = tf.constant(.001, name='lambda')

"""
u_norm = tf.reduce_sum(tf.abs(U, name='user_abs'), name='user_norm') / tf.cast(tf.shape(U)[0], tf.float32)
d_norm = tf.reduce_sum(tf.abs(D, name='director_abs'), name='director_norm')/ tf.cast(tf.shape(D)[1], tf.float32)
t_norm = tf.reduce_sum(tf.abs(T, name='topic_abs'), name='topic_norm')/ tf.cast(tf.shape(T)[1], tf.float32)
g_norm = tf.reduce_sum(tf.abs(G, name='genre_abs'), name='genre_norm')/ tf.cast(tf.shape(G)[1], tf.float32)
c_norm = tf.reduce_sum(tf.abs(C, name='cast_abs'), name='cast_norm')/ tf.cast(tf.shape(C)[1], tf.float32)
"""
u_norm = tf.reduce_sum(tf.abs(U, name='user_abs'), name='user_norm') 
d_norm = tf.reduce_sum(tf.abs(D, name='director_abs'), name='director_norm')
t_norm = tf.reduce_sum(tf.abs(T, name='topic_abs'), name='topic_norm')
g_norm = tf.reduce_sum(tf.abs(G, name='genre_abs'), name='genre_norm')
c_norm = tf.reduce_sum(tf.abs(C, name='cast_abs'), name='cast_norm')

norm_sums = u_norm + d_norm + t_norm + g_norm + c_norm
regularizer = tf.multiply(norm_sums, lda, 'regularizer')

In [13]:
#cost = tf.add(base_cost, regularizer)
cost = base_cost + regularizer

In [14]:
lr = tf.constant(.001, name='learning_rate')
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(lr, global_step, 10000, 0.96, staircase=True)
#learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_step = optimizer.minimize(cost, global_step=global_step)

In [15]:
"""
Generate Training RMSE
"""
g_user_test = g_user
g_item_test = g_movie
g_rate_test = g_rating
g_result_flatten = g_result_flatten
#genre rmse
g_test = tf.gather(g_result_flatten, g_user_test * tf.shape(g_result)[1] + g_item_test, name='g_test')
g_rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(g_rate_test, g_test))))

d_user_test = d_user
d_item_test = d_movie
d_rate_test = d_rating
d_result_flatten = d_result_flatten
#genre rmse
d_test = tf.gather(d_result_flatten, d_user_test * tf.shape(d_result)[1] + d_item_test, name='d_test')
d_rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(d_rate_test, d_test))))

t_user_test = t_user
t_item_test = t_movie
t_rate_test = t_rating
t_result_flatten = t_result_flatten
#genre rmse
t_test = tf.gather(t_result_flatten, t_user_test * tf.shape(t_result)[1] + t_item_test, name='t_test')
t_rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(t_rate_test, t_test))))

c_user_test = c_user
c_item_test = c_movie
c_rate_test = c_rating
c_result_flatten = c_result_flatten
#genre rmse
c_test = tf.gather(c_result_flatten, c_user_test * tf.shape(c_result)[1] + c_item_test, name='c_test')
c_rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(c_rate_test, c_test))))


rmse = tf.divide((g_rmse + d_rmse + t_rmse + c_rmse), 4)



In [16]:
"""
Generate Validation RMSE
"""
g_user_v = g_user_v
g_item_v = g_movie_v
g_rate_v = g_rating_v
g_result_flatten = g_result_flatten
#genre rmse
g_v = tf.gather(g_result_flatten, g_user_v * tf.shape(g_result)[1] + g_item_v, name='g_v')
g_rmse_v = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(g_rate_v, g_v))))

d_user_v = d_user_v
d_item_v = d_movie_v
d_rate_v = d_rating_v
d_result_flatten = d_result_flatten
#genre rmse
d_v = tf.gather(d_result_flatten, d_user_v * tf.shape(d_result)[1] + d_item_v, name='d_v')
d_rmse_v = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(d_rate_v, d_v))))

t_user_v = t_user_v
t_item_v = t_movie_v
t_rate_v = t_rating_v
t_result_flatten = t_result_flatten
#genre rmse
t_v = tf.gather(t_result_flatten, t_user_v * tf.shape(t_result)[1] + t_item_v, name='t_v')
t_rmse_v = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(t_rate_v, t_v))))

c_user_v = c_user_v
c_item_v = c_movie_v
c_rate_v = c_rating_v
c_result_flatten = c_result_flatten
#genre rmse
c_v = tf.gather(c_result_flatten, c_user_v * tf.shape(c_result)[1] + c_item_v, name='c_v')
c_rmse_v = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(c_rate_v, c_v))))


rmse_v = tf.divide((g_rmse_v + d_rmse_v + t_rmse_v + c_rmse_v), 4)

In [17]:
for i in range(len(g_user_v)):
    if g_user_v[i] < 0:
        print(i, g_item_v[i], g_user_v[i])
        break

In [18]:
#training_trend_f = open('../data/'+folder+'/trends.csv', 'w')
#csv_trend = csv.writer(training_trend_f)
#headers = ['round','train_total', 'train_g', 'train_d', 'train_t', 'train_c', 'valid_total', 'valid_g', 'valid_d', 'valid_t', 'valid_c']
#csv_trend.writerow(headers)
lowest_num = 0
lowest_rmse = 9999999
not_dec_num = 0

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
for i in range(100001):
    _, c, r = sess.run([training_step, cost, regularizer])
    if i%20 == 0:
        _rmse_v, _g_rmse_v, _d_rmse_v, _t_rmse_v, _c_rmse_v  = sess.run([rmse_v, g_rmse_v, d_rmse_v, t_rmse_v, c_rmse_v])
        sess.run(clip)
        if lowest_rmse > _rmse_v + 0.001:
            lowest_rmse = _rmse_v
            lowest_num = i
            not_dec_num = 0
        else:
            not_dec_num += 1
            if not_dec_num > 10:
                break
            
        print(i, c, "current rmsr: ", _rmse_v,  "lowest rmse: ", lowest_rmse, "lowest num: ", lowest_num)
        print(_g_rmse_v, _d_rmse_v, _t_rmse_v, _c_rmse_v)
        _rmse, _g_rmse, _d_rmse, _t_rmse, _c_rmse = sess.run([rmse, g_rmse, d_rmse, t_rmse, c_rmse])
        #w_row = [i, _rmse, _g_rmse, _d_rmse, _t_rmse, _c_rmse, _rmse_v, _g_rmse_v, _d_rmse_v, _t_rmse_v, _c_rmse_v]
        #csv_trend.writerow(w_row)
    if i%200 == 0:
        print()
        print("Traing RMSR(total, g, d, t, c):", i , _rmse, _g_rmse, _d_rmse, _t_rmse, _c_rmse)
        print()
#training_trend_f.close()

0 143470.0 current rmsr:  4.01684 lowest rmse:  4.01684 lowest num:  0
4.02873 3.9755 4.05321 4.00992

Traing RMSR(total, g, d, t, c): 0 3.12561 3.19357 3.15693 3.07146 3.0805

20 56726.5 current rmsr:  1.68831 lowest rmse:  1.68831 lowest num:  20
1.3654 1.76819 1.41574 2.20393
40 31340.5 current rmsr:  1.09548 lowest rmse:  1.09548 lowest num:  40
0.904285 1.09524 0.942372 1.44004
60 24985.0 current rmsr:  0.930218 lowest rmse:  0.930218 lowest num:  60
0.868904 0.904794 0.836008 1.11116
80 23094.2 current rmsr:  0.875345 lowest rmse:  0.875345 lowest num:  80
0.866982 0.842984 0.81151 0.979904
100 22326.4 current rmsr:  0.852054 lowest rmse:  0.852054 lowest num:  100
0.8689 0.817075 0.805561 0.916678
120 21897.2 current rmsr:  0.839378 lowest rmse:  0.839378 lowest num:  120
0.870093 0.802851 0.804033 0.880535
140 21619.1 current rmsr:  0.83131 lowest rmse:  0.83131 lowest num:  140
0.870133 0.79357 0.803698 0.857839
160 21386.0 current rmsr:  0.825472 lowest rmse:  0.825472 lowest

In [19]:
movie_feature_dict = pickle.load(open('../data/'+folder+'/movie_feature_dict.pkl', 'rb'))

In [20]:
def generate_matrix(row_num, col_num, user, movie, rating):
    # row is movie infor num
    # col is user num
    matr = [[0 for i in range(col_num)] for j in range(row_num)]
    for i in range(len(user)):
        matr[movie[i]][user[i]] = float(rating[i])
    
    return matr
        
    

In [21]:
user_v, director_v, topic_v, genre_v, cast_v = sess.run([U,D,T,G,C])

d_mat = generate_matrix(DIRECTOR_NUM, USER_NUM, d_user, d_movie, d_rating)
t_mat = generate_matrix(TOPIC_NUM, USER_NUM, t_user, t_movie, t_rating)
c_mat = generate_matrix(CAST_NUM, USER_NUM, c_user, c_movie, c_rating)
g_mat = generate_matrix(GENRE_NUM, USER_NUM, g_user, g_movie, g_rating)


In [22]:
len(d_mat)

194

In [23]:
id_2_genre = pickle.load(open('../data/'+folder+'/id_2_genre.pkl', 'rb'))
id_2_director = pickle.load(open('../data/'+folder+'/id_2_director.pkl', 'rb'))
id_2_cast = pickle.load(open('../data/'+folder+'/id_2_cast.pkl', 'rb'))
genre_2_id = pickle.load(open('../data/'+folder+'/genre_2_id.pkl', 'rb'))
director_2_id = pickle.load(open('../data/'+folder+'/director_2_id.pkl', 'rb'))
cast_2_id = pickle.load(open('../data/'+folder+'/cast_2_id.pkl', 'rb'))

In [24]:

def get_rating_list(info_id_list, info_matrix, user_vector, user_id, matrix):
    result_list = []
    for _id in info_id_list:
        if matrix[_id][user_id] > 0:
            result_list.append(matrix[_id][user_id]) 
        else:
            
            _v = info_matrix[:, int(_id)]
            _rating = np.dot(user_vector, _v)
            result_list.append(_rating)
    return result_list

def get_rating_list_train(id_tup_list, matrix):
    result_list = []
    for tup in id_tup_list:
        user_id = tup[0]
        info_id = tup[1]
        result_list.append(matrix[info_id][user_id])

def gen_regr_data(m_id, user_rating_tups, is_training = True):
    x_train = []
    y_train = []
    user_id_train = []
    for tup in user_rating_tups:
        one_user_x = []
        rating = float(tup[1])
        y_train.append(rating)
        
        user_id = int(tup[0])
        user_id_train.append(user_id)
        feature_dict = movie_feature_dict[m_id]
        u_v = user_v[user_id]
        
        d_x = get_rating_list(feature_dict['d'], director_v, u_v,user_id, d_mat)
        g_x = get_rating_list(feature_dict['g'], genre_v, u_v, user_id, g_mat)
        t_x = get_rating_list(feature_dict['t'], topic_v, u_v, user_id, t_mat)
        c_x = get_rating_list(feature_dict['c'], cast_v, u_v, user_id, c_mat)
        #print(user_id, d_x, g_x, t_x, c_x)
        
        
        one_user_x = d_x + g_x + t_x + c_x
        #one_user_x = d_x + g_x + c_x
        x_train.append(one_user_x)
        
        feature_name = []
        feature_name += ["d_" + id_2_director[_id] for _id in feature_dict['d']]
        feature_name += ["g_" + id_2_genre[_id] for _id in feature_dict['g']]
        feature_name += ["t_" + str(_id) for _id in feature_dict['t']]
        feature_name += ["c_" + id_2_cast[_id] for _id in feature_dict['c']]
        feature_name += ["bias"]
        
        
    return x_train, y_train, feature_name, user_id_train
        
            
              

In [25]:
user_count_list = pickle.load(open('../data/'+folder+'/user_count_list.pkl', 'rb'))

In [26]:
name_list = ['d', 't', 'c', 'g']
divid_list = []
for name in name_list:
    summ = 0
    total = 0
    for user in user_count_list[name]:
        for key in user_count_list[name][user]:
            summ += user_count_list[name][user][key]
            total += 1

    divid_list.append(summ/total)

# Store result matrix

In [27]:
_d_result, _t_result, _g_result, _c_result =  sess.run([d_result, t_result, g_result, c_result])

In [28]:
pickle.dump(_d_result, open('../data/'+folder+'/ranking_matrix_director.pkl', 'wb'))
pickle.dump(_t_result, open('../data/'+folder+'/ranking_matrix_type.pkl', 'wb'))
pickle.dump(_g_result, open('../data/'+folder+'/ranking_matrix_genre.pkl', 'wb'))
pickle.dump(_c_result, open('../data/'+folder+'/ranking_matrix_cast.pkl', 'wb'))

In [29]:
from sklearn import linear_model
import numpy as np

def select_feature_index(feature, n):
    index_list = [0 for i in range(len(feature))]
    temp_f = [abs(feature[i]) for i in range(len(feature))]
    for i in (range(n)):
        min_index = temp_f.index(max(temp_f))
        index_list[min_index] = 1
        temp_f[min_index] = 0
    
    return index_list

def get_predict_rating_old(user_rating, features):
    return np.mean([user_rating[i] + features[i] for i in range(len(user_rating))])


def get_predict_rating(user_rating, features):
    weight_n = [float(abs(ur)) for ur in user_rating]
    summ = sum(weight_n)
    weight = [ur/summ for ur in weight_n]
    #weight = [1./len(weight_n) for ur in weight_n]
    
    weight_total = sum([weight[i]* (user_rating[i] + features[i]) for i in range(len(weight))])
    
    return weight_total

def get_predict_rating_select(user_rating, features, n):
    if len(user_rating) < n:
        return get_predict_rating_old(user_rating, features)
    else:
        index_list = select_feature_index(user_rating, n)
        account_list = []
        for i in range(len(user_rating)):
            if index_list[i] == 1:
                account_list.append(user_rating[i] + features[i])
        return np.mean(account_list)

def get_predict_rating_count(user_rating, features, user_id, feature_name):
    type_prop = {}
    weight = []
    weight_sum = 0
    offset = 0.01
    for j in range(len(feature_name)-1):
        name = feature_name[j]
        dict_name = name.split('_')[0]
        feature_n = name.split('_')[1]
        try:
            if dict_name == 'd':
                feature_key = director_2_id[feature_n]
                offset = 1/divid_list[0]
            elif dict_name == 'c':
                feature_key = cast_2_id[feature_n]
                offset = 1/divid_list[2]
            elif dict_name == 'g':
                feature_key = genre_2_id[feature_n]
                offset = 1/divid_list[3]
            elif dict_name == 't':
                feature_key = int(feature_n)
                offset = 1/divid_list[1]
        except:
            feature_key = ''
        try:
            prop = user_count_list[dict_name][user_id][feature_key]
        except:
            prop = 1
        prop = offset * prop
        weight_sum += prop
        weight.append(prop)      
    
    weight = [w/float(weight_sum) for w in weight]
    result = 0
    ppr_list = []
    for i in range(len(user_rating)):
        ppr = (user_rating[i] + features[i]) * weight[i]
        ppr_list.append(ppr)
        result += ppr
        # add each kind information proportion to calculate average proportion of each information in movie 
    
    for i in range(len(ppr_list)):
        name = feature_name[i]
        dict_name = name.split('_')[0]
        if dict_name not in type_prop:
            type_prop[dict_name] = [ppr_list[i]/result]
        else:
            type_prop[dict_name] = type_prop[dict_name] + [ppr_list[i]/result]
    """
    print(user_id)
    for i in range(len(weight)):
        print(feature_name[i], end='')
        print(" %.2f %.2f %.2f" % (user_rating[i], features[i], weight[i]))
    
    """
    type_mean = {}
    for key in type_prop:
        type_mean[key] = np.sum(type_prop[key])
    
    return result, type_mean
    

# Equal weight

In [30]:
total_predict = np.array([])
total_rating = np.array([])
total_predict_v = np.array([])
total_rating_v = np.array([])
total_predict_tr = np.array([])
total_rating_tr = np.array([])
#train_rmse_compare_F = []
for m in range(0, MOVIE_NUM+1):
    if m not in movie_rating:
        print("not in:", m)
        continue
    user_rating_tups = movie_rating[m]
    x_train, y_train, feature_name, user_id_train = gen_regr_data(int(m), user_rating_tups)
    
    if len(y_train) < 5:
        print('less than 5:', m)
        continue
    #x_train = x_train[1:2]
    #y_train = y_train[1:2]
    
    #if len(y_train) > 50:
     #   continue
    
    if m in movie_rating_test:     
        user_rating_tups_test = movie_rating_test[m]
        x_test, y_test, _, user_id_test= gen_regr_data(int(m), user_rating_tups_test)
    else:
        x_test = []
        y_test = []
        user_id_test = []
    
    if m in movie_rating_validation:     
        user_rating_tups_validation = movie_rating_validation[m]
        x_valid, y_valid, _, user_id_valid= gen_regr_data(int(m), user_rating_tups_validation)
    else:
        x_valid = []
        y_valid = []
        user_id_valid = []
        
    print(len(x_train), len(x_test), len(x_valid))
    feature = [0 for i in range(len(feature_name)-1)]
    
    for j in range(len(feature_name)-1):
        name = feature_name[j]
        dict_name = name.split('_')[0]
        feature_n = name.split('_')[1]

        
        for i in range(len(x_train)):
            feature[j] += (y_train[i] - x_train[i][j])
        
        feature[j] /= len(x_train)
    
    #weight = [1 - abs(f) for f in feature] if len(feature) > 0 else []
    #select_indexs = select_feature_index(feature, 100)
    #select_indexs = [1 for i in range(len(feature))]
    #for i in range(len(feature)):
        #print(feature_name[i], 1/(1-feature[i]))
        #if select_indexs[i] == 1:
            #print(feature_name[i], feature[i], weight[i])
    #print()
    

    predict_t = []
    predict_v = []
    predict_tr = []
    
    for i in range(len(x_train)):
        cc = get_predict_rating_old(x_train[i], feature)
        #print("%.2f %.2f" %(cc, y_train[i]))
        #print()
        predict_tr.append(cc)
    
    for i in range(len(x_test)):
        cc = get_predict_rating_old(x_test[i], feature)
        #print("%.2f %.2f" %(cc, y_test[i]))
        #print()
        predict_t.append(cc)
    

    for i in range(len(x_valid)):
        cc = get_predict_rating_old(x_valid[i], feature)
        predict_v.append(cc)
        #print("%.2f %.2f" %(cc, y_valid[i]))
        #print()
     
    train_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_tr, y_train))))
    valid_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_v, y_valid))))
    test_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_t, y_test))))
    
    #print(m, train_rmse, valid_rmse, test_rmse)
    #train_rmse_compare_F.append([m, len(y_train), len(y_valid),len(y_test), train_rmse, valid_rmse, test_rmse])
    
    total_predict = np.concatenate((total_predict, predict_t), axis = 0)
    total_rating = np.concatenate((total_rating, y_test), axis = 0)
    total_predict_v = np.concatenate((total_predict_v, predict_v), axis = 0)
    total_rating_v = np.concatenate((total_rating_v, y_valid), axis = 0)
    total_predict_tr = np.concatenate((total_predict_tr, predict_tr), axis = 0)
    total_rating_tr = np.concatenate((total_rating_tr, y_train), axis = 0)
    
    #print(feature)
    #print(feature_name)
        
    

1427 175 172
759 107 88
1106 148 163
543 72 80
208 28 26
692 71 84
519 57 61
1002 132 151
371 42 51
1119 151 165
200 24 24
1168 134 132
352 45 47
378 53 59
1000 131 89
530 51 52
1366 157 160
873 104 104
273 38 33
539 86 58
383 51 41
1104 125 136
868 122 115
516 53 63
787 89 99
313 39 31
639 72 85
346 36 54
480 44 69
193 20 25
50 7 5
109 14 20
238 33 25
13 3 0
176 23 23
657 70 88
412 55 50
53 7 3
289 36 42
335 45 40
1258 142 165


/Users/Suyixin/anaconda/envs/ws/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/Suyixin/anaconda/envs/ws/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


135 17 16
789 104 104
1149 139 154
1330 173 178
912 102 125
493 55 49
97 15 12
742 84 84
1136 138 156
43 6 13
642 69 84
164 13 17
1242 155 186
1045 123 114
810 92 103
less than 5: 56
265 36 32
235 35 30
1560 207 202
410 54 52
46 6 7
322 37 44
31 7 3
32 4 7
250 42 28
1177 165 150
61 7 12
300 37 45
less than 5: 69
254 33 32
217 29 38
374 54 51
16 4 1
447 53 54
731 87 81
743 95 90
664 92 79
66 7 14
71 5 12
335 50 41
113 13 14
231 26 35
70 3 9
9 0 1
1009 110 118
38 3 2
37 4 7
648 70 72
748 108 84
90 11 12
58 8 9
666 83 84
186 37 28
71 8 15
266 43 33
445 36 48
1070 133 109
440 54 56
240 24 29
271 29 29
535 77 64
58 7 11
402 46 54
96 14 10
244 32 27
94 10 11
275 27 39
less than 5: 108
117 20 13
133 16 12
212 30 21
103 9 10
387 43 43
165 19 21
89 13 19
195 23 25
131 14 9
367 61 44
488 58 62
105 14 17
663 88 73
895 105 114
55 4 4
50 9 8
216 30 30
728 106 77
571 73 68
1064 130 146
93 13 13
632 91 95
244 27 32
60 7 9
86 17 11
331 35 35
44 6 2
1224 168 173
219 40 26
26 5 3
138 16 9
239 25 29
111 

In [31]:
rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict, total_rating))))  
print("train rmse:", rmse)

train rmse: 0.760961253976


In [32]:
for i in range(len(total_predict)):
    if total_predict[i] < 0:
        total_predict[i] = 0.
    if total_predict[i] > 5:
        total_predict[i] = 5.

for i in range(len(total_predict_v)):
    if total_predict_v[i] < 0:
        total_predict_v[i] = 0.
    if total_predict_v[i] > 5:
        total_predict_v[i] = 5.
        
for i in range(len(total_predict_tr)):
    if total_predict_tr[i] < 0:
        total_predict_tr[i] = 0.
    if total_predict_tr[i] > 5:
        total_predict_tr[i] = 5.

        
rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict_tr, total_rating_tr))))  
print("train rmse:", rmse)

mae = mean_absolute_error(total_rating_tr, total_predict_tr)
print("train mae:", mae)

rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict_v, total_rating_v))))  
print("valid rmse:", rmse)

mae = mean_absolute_error(total_rating_v, total_predict_v)
print("valid mae:", mae)

rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict, total_rating))))  
print("test rmse:", rmse)

mae = mean_absolute_error(total_rating, total_predict)
print("test mae:", mae)
        


train rmse: 0.29143128588
train mae: 0.21081254878
valid rmse: 0.753641973129
valid mae: 0.554399160574
test rmse: 0.760657148598
test mae: 0.555822446024


# weight with number of featured movies

In [33]:
total_predict = np.array([])
total_rating = np.array([])
total_predict_v = np.array([])
total_rating_v = np.array([])
total_predict_tr = np.array([])
total_rating_tr = np.array([])
train_rmse_compare_E = []
movie_feature = {}
feature_name_dict = {}
dropped_movie = 0
info_prop = {}
info_prop['t'] = 0
info_prop['g'] = 0
info_prop['c'] = 0
info_prop['d'] = 0
for m in range(0, MOVIE_NUM+1):
    if m not in movie_rating:
        print('not in', m)
        dropped_movie += 1
        continue
    user_rating_tups = movie_rating[m]
    x_train, y_train, feature_name, user_id_train = gen_regr_data(int(m), user_rating_tups)
    
    if len(y_train) < 0:
        print('less than 5:', m)
        dropped_movie += 1
        continue
    #if len(y_train) > 50:
     #   continue
    
    if m in movie_rating_test:     
        user_rating_tups_test = movie_rating_test[m]
        x_test, y_test, _, user_id_test= gen_regr_data(int(m), user_rating_tups_test)
    else:
        x_test = []
        y_test = []
        user_id_test = []
    
    if m in movie_rating_validation:     
        user_rating_tups_validation = movie_rating_validation[m]
        x_valid, y_valid, _, user_id_valid= gen_regr_data(int(m), user_rating_tups_validation)
    else:
        x_valid = []
        y_valid = []
        user_id_valid = []
    
    if len(x_test) == 0:
        continue
        
    
    feature = [0 for i in range(len(feature_name)-1)]
    
    for j in range(len(feature_name)-1):
        prop_sum = 0
        name = feature_name[j]
        dict_name = name.split('_')[0]
        feature_n = name.split('_')[1]
        offset = 0.01
        try:
            if dict_name == 'd':
                feature_key = director_2_id[feature_n]
                offset = 1/divid_list[0]
            elif dict_name == 'c':
                feature_key = cast_2_id[feature_n]
                offset = 1/divid_list[2]
            elif dict_name == 'g':
                feature_key = genre_2_id[feature_n]
                offsedivid_listt = 1/divid_list[3]
            elif dict_name == 't':
                feature_key = int(feature_n)
                offset = 1/divid_list[1]
        except:
            feature_key = ''
        
        for i in range(len(x_train)):
            try:
                prop = user_count_list[dict_name][user_id_train[i]][feature_key]
            except:
                prop = 1
            prop = offset * prop
            #prop_sum += prop
            #feature[j] += (y_train[i] - x_train[i][j]) * prop
            prop_sum += 1
            feature[j] += (y_train[i] - x_train[i][j])
        
        feature[j] /= prop_sum
    feature_name_dict[m] = feature_name
    movie_feature[m] = feature
    #weight = [1 - abs(f) for f in feature] if len(feature) > 0 else []
    #select_indexs = select_feature_index(feature, 100)
    #select_indexs = [1 for i in range(len(feature))]
    #for i in range(len(feature)):
        #print(feature_name[i], 1/(1-feature[i]))
        #if select_indexs[i] == 1:
            #print(feature_name[i], feature[i], weight[i])
    #print()
    

    predict_t = []
    predict_v = []
    predict_tr = []
    for i in range(len(x_train)):
        cc,info_prop_s = get_predict_rating_count(x_train[i], feature, user_id_train[i], feature_name)
        #print("%.2f %.2f" %(cc, y_train[i]))
        #print()
        for key in info_prop:
            if key in info_prop_s:
                info_prop[key] = info_prop[key] + info_prop_s[key]
        predict_tr.append(cc)
    
    for i in range(len(x_test)):
        cc,info_prop_s = get_predict_rating_count(x_test[i], feature, user_id_test[i], feature_name)
        #print("%.2f %.2f" %(cc, y_test[i]))
        #print()
        for key in info_prop:
            if key in info_prop_s:
                info_prop[key] = info_prop[key] + info_prop_s[key]
        predict_t.append(cc)
    

    for i in range(len(x_valid)):
        cc,info_prop_s = get_predict_rating_count(x_valid[i], feature, user_id_valid[i], feature_name )
        for key in info_prop:
            if key in info_prop_s:
                info_prop[key] = info_prop[key] + info_prop_s[key]
        predict_v.append(cc)
        #print("%.2f %.2f" %(cc, y_valid[i]))
        #print()
        
        
    #train_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_tr, y_train))))
    #valid_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_v, y_valid))))
    #test_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_t, y_test))))
    
    #print(m, train_rmse, valid_rmse, test_rmse)
    
    #print(m, train_rmse, valid_rmse, test_rmse)
    #train_rmse_compare_E.append([m, len(y_train), len(y_valid),len(y_test), train_rmse, valid_rmse, test_rmse])
    
        
    total_predict = np.concatenate((total_predict, predict_t), axis = 0)
    total_rating = np.concatenate((total_rating, y_test), axis = 0)
    total_predict_v = np.concatenate((total_predict_v, predict_v), axis = 0)
    total_rating_v = np.concatenate((total_rating_v, y_valid), axis = 0)
    total_predict_tr = np.concatenate((total_predict_tr, predict_tr), axis = 0)
    total_rating_tr = np.concatenate((total_rating_tr, y_train), axis = 0)


not in 186


In [34]:
len(total_predict)

9932

In [35]:
summ = 0
for key in info_prop:
    summ += info_prop[key]

for key in info_prop:
    print(key, ':', info_prop[key]/summ)

t : 0.101275083226
g : 0.522117231551
c : 0.275120535756
d : 0.101487149467


In [36]:
index = 3
for i in range(len(movie_feature[index])):
    print(feature_name_dict[index][i], movie_feature[index][i])

d_Michael Bay 0.03360957642725598
g_Action -0.4520801335041177
g_Adventure -0.5222022198128377
g_Sci-Fi -0.5247564581382188
t_0 -0.07597237047091458
c_Shia LaBeouf -0.04929876753081174
c_Rosie Huntington-Whiteley -7.68773217788322e-17
c_Josh Duhamel 0.0


In [37]:
for i in range(len(total_predict)):
    if total_predict[i] < 0:
        total_predict[i] = 0.
    if total_predict[i] > 5:
        total_predict[i] = 5.

for i in range(len(total_predict_v)):
    if total_predict_v[i] < 0:
        total_predict_v[i] = 0.
    if total_predict_v[i] > 5:
        total_predict_v[i] = 5.
        
for i in range(len(total_predict_tr)):
    if total_predict_tr[i] < 0:
        total_predict_tr[i] = 0.
    if total_predict_tr[i] > 5:
        total_predict_tr[i] = 5.

        
rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict_tr, total_rating_tr))))  
print( rmse)

mae = mean_absolute_error(total_rating_tr, total_predict_tr)
print(mae)

rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict_v, total_rating_v))))  
print(rmse)

mae = mean_absolute_error(total_rating_v, total_predict_v)
print( mae)


rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict, total_rating))))  
print( rmse)

mae = mean_absolute_error(total_rating, total_predict)
print(mae)
        


0.402485198126
0.288492403305
0.75007781213
0.552879152063
0.757359215551
0.554899368944


# Get one user's information

In [38]:
user_id = 5
feature_dict = ['d','t','c', 'g']
one_user_count_list = {}
for type_name in feature_dict:
    one_user_count_list[type_name] = {}
for i in range(len(feature_dict)):
    dict_name = feature_dict[i]
    divid_value = divid_list[i]
    for feature_key in user_count_list[dict_name][user_id]:
        one_user_count_list[dict_name][feature_key] = user_count_list[dict_name][user_id][feature_key]/divid_value
    

In [39]:
user_movie_rating = {}
user_movie_info = {}

for m in range(0, MOVIE_NUM+1):
    if m not in movie_rating:
        print('not in', m)
        dropped_movie += 1
        continue
    user_rating_tups = movie_rating[m]
    x_train, y_train, feature_name, user_id_train = gen_regr_data(int(m), user_rating_tups)
    if len(y_train) < 5:
        print('less than 5:', m)
        dropped_movie += 1
        continue
    
    one_uesr_ratings = []
    one_user_info = []
    weight = []
    x_pred, _, feature_name, user_id_pred = gen_regr_data(int(m), [(user_id, 0)])
    if user_id in user_id_train:
        ind = user_id_train.index(user_id)
        real_rating = y_train[ind]
    else:
        real_rating = -1
    for j in range(len(feature_name)-1):
        prop_sum = 0
        name = feature_name[j]
        dict_name = name.split('_')[0]
        feature_n = name.split('_')[1]
        offset = 0.01
        try:
            if dict_name == 'd':
                feature_key = director_2_id[feature_n]
            elif dict_name == 'c':
                feature_key = cast_2_id[feature_n]
            elif dict_name == 'g':
                feature_key = genre_2_id[feature_n]
            elif dict_name == 't':
                feature_key = int(feature_n)
        except:
            feature_key = ''
        try:
            weight.append(one_user_count_list[dict_name][feature_key])
        except:
            # if user have never seen this information
            weight.append(0.1)

    weight_normal = [w/sum(weight) for w in weight]
    #print('begin')
    try:
        one_pred_value = sum([weight_normal[i]*(x_pred[0][i] + movie_feature[m][i]) for i in range(len(x_pred[0]))])
    except:
        continue
    feature_name_dict
    """
    print("movie:", m)
    print("pred:", one_pred_value)
    print("real:", real_rating)
    
    for i in range(len(x_pred[0])):
        print(feature_name_dict[m][i], x_pred[0][i], movie_feature[m][i], weight_normal[i])
    """
    user_movie_rating[m] = (one_pred_value, real_rating)
    user_movie_info[m] = {}
    user_movie_info[m]['info_name'] = feature_name_dict[m][0:-1]
    user_movie_info[m]['info_rating'] = x_pred[0]
    user_movie_info[m]['movie_offset'] = movie_feature[m]
    user_movie_info[m]['weight'] = weight_normal
    



        

less than 5: 56
less than 5: 69
less than 5: 108
less than 5: 171
not in 186
less than 5: 192


In [40]:
"""
for m in user_movie_rating:
    if user_movie_rating[m][1] > 0:
        print(m, user_movie_rating[m][1])
        length = len(user_movie_info[m]['info_rating'])
        for i in range((length)):
            print(user_movie_info[m]['info_name'][i], user_movie_info[m]['info_rating'][i] + user_movie_info[m]['movie_offset'][i], user_movie_info[m]['weight'][i])

"""
        

"\nfor m in user_movie_rating:\n    if user_movie_rating[m][1] > 0:\n        print(m, user_movie_rating[m][1])\n        length = len(user_movie_info[m]['info_rating'])\n        for i in range((length)):\n            print(user_movie_info[m]['info_name'][i], user_movie_info[m]['info_rating'][i] + user_movie_info[m]['movie_offset'][i], user_movie_info[m]['weight'][i])\n\n"

In [41]:
best_movie = -1
best_rating = 0
for m in user_movie_rating:
    if user_movie_rating[m][1] < 0 and user_movie_rating[m][0] > best_rating:
        best_rating = user_movie_rating[m][0]
        best_movie = m

best_movie

121

In [42]:
best_movie = 1817
print('rating:',user_movie_rating[best_movie])
length = len(user_movie_info[best_movie]['info_rating'])
for i in range((length)):
    part = (user_movie_info[best_movie]['info_rating'][i] + user_movie_info[best_movie]['movie_offset'][i]) * user_movie_info[best_movie]['weight'][i]
    print(user_movie_info[best_movie]['info_name'][i], user_movie_info[best_movie]['info_rating'][i], user_movie_info[best_movie]['movie_offset'][i], user_movie_info[best_movie]['weight'][i], part)
    #print(user_movie_info[best_movie]['info_name'][i])
    
    #print(part)


KeyError: 1817

In [ ]:
length

## test cold start

In [ ]:
def get_predict_rating_0(user_rating, features):
    return np.mean([user_rating[i] for i in range(len(user_rating))])

In [ ]:
total_predict = np.array([])
total_rating = np.array([])
total_predict_v = np.array([])
total_rating_v = np.array([])
total_predict_tr = np.array([])
total_rating_tr = np.array([])
train_rmse_compare_E = []
movie_feature = {}
feature_name_dict = {}
dropped_movie = 0
total_predicted = 0
info_prop = {}
info_prop['t'] = 0
info_prop['g'] = 0
info_prop['c'] = 0
info_prop['d'] = 0
largest_train = 10
_RMSE_list = []
for largest_train in range(0, 200, 3):
    for m in range(0, MOVIE_NUM+1):
        have_train = False
        x_train = []
        y_train = []
        if m in movie_rating:
            user_rating_tups = movie_rating[m]
            x_train, y_train, feature_name, user_id_train = gen_regr_data(int(m), user_rating_tups)
            have_train = True

        if len(y_train) > largest_train:
            #print('larger:', m)
            #dropped_movie += 1
            continue
        #if len(y_train) > 50:
         #   continue

        if m in movie_rating_test:     
            user_rating_tups_test = movie_rating_test[m]
            x_test, y_test, _, user_id_test= gen_regr_data(int(m), user_rating_tups_test)
        else:
            x_test = []
            y_test = []
            user_id_test = []
        """
        if m in movie_rating_validation:     
            user_rating_tups_validation = movie_rating_validation[m]
            x_valid, y_valid, _, user_id_valid= gen_regr_data(int(m), user_rating_tups_validation)
        else:
            x_valid = []
            y_valid = []
            user_id_valid = []
        """
        if len(x_test) == 0:
            continue


        feature = [1.0/(len(feature_name)-1) for i in range(len(feature_name)-1)]

        for j in range(len(feature_name)-1):
            prop_sum = 0
            name = feature_name[j]
            dict_name = name.split('_')[0]
            feature_n = name.split('_')[1]
            offset = 0.01
            try:
                if dict_name == 'd':
                    feature_key = director_2_id[feature_n]
                    offset = 1/divid_list[0]
                elif dict_name == 'c':
                    feature_key = cast_2_id[feature_n]
                    offset = 1/divid_list[2]
                elif dict_name == 'g':
                    feature_key = genre_2_id[feature_n]
                    offsedivid_listt = 1/divid_list[3]
                elif dict_name == 't':
                    feature_key = int(feature_n)
                    offset = 1/divid_list[1]
            except:
                feature_key = ''
            #print(len(x_train))
            for i in range(len(x_train)):
                try:
                    prop = user_count_list[dict_name][user_id_train[i]][feature_key]
                except:
                    prop = 1
                prop = offset * prop
                #prop_sum += prop
                #feature[j] += (y_train[i] - x_train[i][j]) * prop
                prop_sum += 1
                feature[j] += (y_train[i] - x_train[i][j])

            if len(x_train) > 0:
                feature[j] /= prop_sum

        feature_name_dict[m] = feature_name
        movie_feature[m] = feature
        #weight = [1 - abs(f) for f in feature] if len(feature) > 0 else []
        #select_indexs = select_feature_index(feature, 100)
        #select_indexs = [1 for i in range(len(feature))]
        #for i in range(len(feature)):
            #print(feature_name[i], 1/(1-feature[i]))
            #if select_indexs[i] == 1:
                #print(feature_name[i], feature[i], weight[i])
        #print()


        predict_t = []
        predict_v = []
        predict_tr = []

        """
        for i in range(len(x_train)):
            cc,info_prop_s = get_predict_rating_count(x_train[i], feature, user_id_train[i], feature_name)
            #print("%.2f %.2f" %(cc, y_train[i]))
            #print()
            for key in info_prop:
                if key in info_prop_s:
                    info_prop[key] = info_prop[key] + info_prop_s[key]
            predict_tr.append(cc)

        for i in range(len(x_valid)):
            cc,info_prop_s = get_predict_rating_count(x_valid[i], feature, user_id_valid[i], feature_name )
            for key in info_prop:
                if key in info_prop_s:
                    info_prop[key] = info_prop[key] + info_prop_s[key]
            predict_v.append(cc)
        """
        for i in range(len(x_test)):
            if have_train:
                cc,info_prop_s = get_predict_rating_count(x_test[i], feature, user_id_test[i], feature_name)
            else:
                cc = get_predict_rating_0(x_test[i], feature)
            #print("%.2f %.2f" %(cc, y_test[i]))
            #print()
            total_predicted += 1
            predict_t.append(cc)

        #train_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_tr, y_train))))
        #valid_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_v, y_valid))))
        #test_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_t, y_test))))

        #print(m, train_rmse, valid_rmse, test_rmse)

        #print(m, train_rmse, valid_rmse, test_rmse)
        #train_rmse_compare_E.append([m, len(y_train), len(y_valid),len(y_test), train_rmse, valid_rmse, test_rmse])


        total_predict = np.concatenate((total_predict, predict_t), axis = 0)
        total_rating = np.concatenate((total_rating, y_test), axis = 0)
        total_predict_v = np.concatenate((total_predict_v, predict_v), axis = 0)
        total_rating_v = np.concatenate((total_rating_v, y_valid), axis = 0)
        total_predict_tr = np.concatenate((total_predict_tr, predict_tr), axis = 0)
        total_rating_tr = np.concatenate((total_rating_tr, y_train), axis = 0)
    rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict, total_rating)))) 
    _RMSE_list.append((largest_train, rmse))
    print(largest_train)

In [ ]:
rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict, total_rating))))  
print( rmse)

In [ ]:
len(total_rating)

In [ ]:
_RMSE_list

In [ ]:
import csv
with open('MMF_RMSE.csv','w') as f:
    f_csv = csv.writer(f)
    headers = ['num, RMSE']
    f_csv.writerow(headers)
    for p in _RMSE_list:
        row = [p[0], p[1]]
        f_csv.writerow(row)